## Experiment 1

Use the feature extractor **output of pretrain model** to produce laten space, then concatenate with the original model
Here we use the VGG pretrain model from CIFAR 10 and the target task is CIFAR10 classification

In [2]:
import zipfile
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
print (session.run(c))

[[22. 28.]
 [49. 64.]]


In [3]:
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras.callbacks import *
from keras.models import *

Using TensorFlow backend.


In [4]:
from keras.layers import *
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras. callbacks import *
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
import time
from keras.applications.vgg16 import VGG16

def define_block(input_layer, nb_layers, nb_neurons, kernel_size=(3,3), batch_normalization=True, activation='relu', **kwargs):
    index = None
    if 'index' in kwargs:
        index = kwargs['index']
    
    for i in range(nb_layers):
        if i == 0:
            b = input_layer
            
        b = Conv2D(nb_neurons, kernel_size=kernel_size, strides=1, padding='same', name='conv' + str(i) + '_block' + str(index))(b)
        if batch_normalization:
            b = BatchNormalization(name='batchnorm' + str(i) + '_block' + str(index))(b)
        b =  Activation(activation, name=activation + str(i) + '_block' + str(index))(b)
    b = MaxPooling2D(pool_size=(2,2), name='maxpooling_block' + str(index))(b)
    return b

def define_skipped_connection(src_layer, dst_layer):
    src_shape = src_layer.output_shape
    dst_shape = dst_layer.input_shape
    

def SkippedVGG(nb_blocks, input_shape, num_classes, nb_layers, nb_neurons, include_top=True, verbose=1, layer_mode=False):
    concats = []
    maxpooling_outputs = []
    
    for i in range(nb_blocks):
        concats.append([])
        
    if type(nb_layers) is int:
        temp = nb_layers
        nb_layers = []
        for i in range(nb_blocks):
            nb_layers.append(temp)
    
    if verbose:
        print('Create DensedVGG model with ' + str(nb_blocks) + ' blocks, input shape = ' + str(input_shape))
    
    for i in range(nb_blocks):
        if verbose:
            print('Create block ' + str(i) + ':')
        
        if i == 0: # First block
            b = Input(shape=input_shape)
            inputs = b
        
        # check all layers before
        if len(concats[i-1]) > 1:
            if verbose:
                print('Concatenate ', concats[i-1])
            b = concatenate(concats[i-1])
        elif len(concats[i-1]) == 1:
            if verbose:
                print('Get direct output from the previous block ', concats[i-1])
            b = concats[i-1][0]
            
        # create main block
        b = define_block(b, nb_layers[i], nb_neurons[i], index=i)
        concats[i].append(b)
        maxpooling_outputs.append(b)
        
        
        # create cropping layer
        for j in range(i+1, nb_blocks):
            if verbose:
                print('-- create skipped connection from block '+ str(i) + ' to block ' + str(j+1) + ' ...')
                
            src_shape = b.get_shape()
            src_shape = (int(src_shape[1]), int(src_shape[2]), int(src_shape[3]))
            dst_shape = (src_shape[0]//2**(j-i), src_shape[1]//2**(j-i), src_shape[2])
            
            print(src_shape, dst_shape)
            h = src_shape[0] - dst_shape[0]
            w = src_shape[1] - dst_shape[1]
            if h % 2 == 0:
                h_0 = h // 2
                h_1 = h // 2
            else:
                h_0 = h // 2
                h_1 = (h // 2) + 1
            
            if w % 2 == 0:
                w_0 = w // 2
                w_1 = w // 2
            else:
                w_0 = w // 2
                w_1 = (w // 2) + 1
                
            concat_layer = Cropping2D(((h_0,h_1),(w_0,w_1)), name='cropping_block' + str(i) + '_block' + str(j+1))(b)
            concats[j].append(concat_layer)
            
    # top model
    if verbose:
        print('Create top model')
    
    if len(concats[nb_blocks-1]) == 1:
        outputs = concats[nb_blocks-1]
    else:
        outputs = concatenate(concats[nb_blocks-1])
        
    if layer_mode:
        return outputs
        
    if include_top:
        outputs = GlobalAveragePooling2D()(outputs)
        outputs = Dense(num_classes)(outputs)
        outputs = BatchNormalization()(outputs)
        outputs = Activation('softmax')(outputs)
        

    model = Model(inputs=inputs, outputs=outputs)
    
    # compile
#     model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    
    if verbose:
        model.summary()
    return model

def SkippedVGG_layer(nb_blocks, input_layer, nb_layers, nb_neurons, verbose=1):
    concats = []
    for i in range(nb_blocks):
        concats.append([])
        
    if type(nb_layers) is int:
        temp = nb_layers
        nb_layers = []
        for i in range(nb_blocks):
            nb_layers.append(temp)
    
    if verbose:
        print('Create DensedVGG model with ' + str(nb_blocks) + ' blocks')
    
    for i in range(nb_blocks):
        if verbose:
            print('Create block ' + str(i) + ':')
        
        if i == 0: # First block
            b = input_layer
            inputs = b
        
        # check all layers before
        if len(concats[i-1]) > 1:
            if verbose:
                print('Concatenate ', concats[i-1])
            b = concatenate(concats[i-1])
        elif len(concats[i-1]) == 1:
            if verbose:
                print('Get direct output from the previous block ', concats[i-1])
            b = concats[i-1][0]
            
        # create main block
        b = define_block(b, nb_layers[i], nb_neurons[i], index=i)
        concats[i].append(b)
        
        # create cropping layer
        for j in range(i+1, nb_blocks):
            if verbose:
                print('-- create skipped connection from block '+ str(i) + ' to block ' + str(j+1) + ' ...')
                
            src_shape = b.get_shape()
            src_shape = (int(src_shape[1]), int(src_shape[2]), int(src_shape[3]))
            dst_shape = (src_shape[0]//2**(j-i), src_shape[1]//2**(j-i), src_shape[2])
            
            print(src_shape, dst_shape)
            h = src_shape[0] - dst_shape[0]
            w = src_shape[1] - dst_shape[1]
            if h % 2 == 0:
                h_0 = h // 2
                h_1 = h // 2
            else:
                h_0 = h // 2
                h_1 = (h // 2) + 1
            
            if w % 2 == 0:
                w_0 = w // 2
                w_1 = w // 2
            else:
                w_0 = w // 2
                w_1 = (w // 2) + 1
                
            concat_layer = Cropping2D(((h_0,h_1),(w_0,w_1)), name='cropping_block' + str(i) + '_block' + str(j+1))(b)
            concats[j].append(concat_layer)
   
    if len(concats[nb_blocks-1]) == 1:
        outputs = concats[nb_blocks-1]
    else:
        outputs = concatenate(concats[nb_blocks-1])
    
    return outputs

In [9]:
input_layer = Input((32,32,3))

pretrain_vgg = VGG16(include_top=False, weights='imagenet')
for layer in pretrain_vgg.layers:
    layer.trainable=False
    
vgg_out = pretrain_vgg(input_layer)

skippedvgg_out = SkippedVGG_layer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512])

vgg_out = GlobalAveragePooling2D()(vgg_out)
skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

concat_layer = concatenate([vgg_out, skippedvgg_out])
outputs = Dense(10)(concat_layer)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

Create DensedVGG model with 3 blocks
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_1/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Concatenate  [<tf.Tensor 'cropping_block0_block2_1/strided_slice:0' shape=(?, 8, 8, 64) dtype=float32>, <tf.Tensor 'maxpooling_block1_1/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________

In [9]:
import pickle

with open('/data/Quan/datasets/cifar10/train_set.hdf5', 'rb') as dt:
    train_set = pickle.load(dt)
    
with open('/data/Quan/datasets/cifar10/validation_set.hdf5', 'rb') as dt:
    validation_set = pickle.load(dt)
    
with open('/data/Quan/datasets/cifar10/test_set.hdf5', 'rb') as dt:
    test_set = pickle.load(dt)
    
train_images = train_set[0]
train_labels = train_set[1]

validation_images = validation_set[0]
validation_labels = validation_set[1]
    
test_images = test_set[0]
test_labels = test_set[1]

batch_size = 32

train_datagen = ImageDataGenerator(rotation_range=360,
                                   rescale=1./255,
                                   fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(train_images, train_labels, batch_size = batch_size, shuffle=True)
validation_generator = test_datagen.flow(validation_images, validation_labels, batch_size=batch_size, shuffle=False)
test_generator = test_datagen.flow(test_images, test_labels, batch_size=1, shuffle=False)

In [19]:
import pickle

epochs = 300

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea1_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea1_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

W0912 10:15:55.402318 140538123249408 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/300
546/546 [==============================] - 22s 41ms/step - loss: 1.5957 - acc: 0.4410 - val_loss: 2.0317 - val_acc: 0.3315
Epoch 2/300
546/546 [==============================] - 20s 37ms/step - loss: 1.3576 - acc: 0.5280 - val_loss: 2.3610 - val_acc: 0.2804
Epoch 3/300
546/546 [==============================] - 19s 35ms/step - loss: 1.2633 - acc: 0.5621 - val_loss: 1.4322 - val_acc: 0.4898
Epoch 4/300
546/546 [==============================] - 19s 35ms/step - loss: 1.1851 - acc: 0.5889 - val_loss: 1.1879 - val_acc: 0.5876
Epoch 5/300
546/546 [==============================] - 19s 35ms/step - loss: 1.1346 - acc: 0.6015 - val_loss: 1.4234 - val_acc: 0.4963
Epoch 6/300
546/546 [==============================] - 19s 36ms/step - loss: 1.0758 - acc: 0.6267 - val_loss: 1.2964 - val_acc: 0.5546
Epoch 7/300
546/546 [==============================] - 19s 36ms/step - loss: 1.0333 - acc: 0.6381 - val_loss: 1.3578 - val_acc: 0.5208
Epoch 8/300
546/546 [==============================] - 

546/546 [==============================] - 20s 36ms/step - loss: 0.0655 - acc: 0.9782 - val_loss: 0.9657 - val_acc: 0.7770
Epoch 122/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0642 - acc: 0.9792 - val_loss: 0.9481 - val_acc: 0.7871
Epoch 123/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0619 - acc: 0.9798 - val_loss: 1.0182 - val_acc: 0.7651
Epoch 124/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0662 - acc: 0.9781 - val_loss: 1.1223 - val_acc: 0.7611
Epoch 125/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0637 - acc: 0.9795 - val_loss: 0.9269 - val_acc: 0.7755
Epoch 126/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0578 - acc: 0.9812 - val_loss: 1.0164 - val_acc: 0.7732
Epoch 127/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0598 - acc: 0.9801 - val_loss: 1.0408 - val_acc: 0.7649
Epoch 128/300
546/546 [==============================] 

Epoch 181/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0367 - acc: 0.9888 - val_loss: 1.0955 - val_acc: 0.7786
Epoch 182/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0391 - acc: 0.9877 - val_loss: 1.0751 - val_acc: 0.7800
Epoch 183/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0370 - acc: 0.9877 - val_loss: 1.1315 - val_acc: 0.7696
Epoch 184/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0368 - acc: 0.9877 - val_loss: 1.0547 - val_acc: 0.7840
Epoch 185/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0348 - acc: 0.9889 - val_loss: 1.1652 - val_acc: 0.7680
Epoch 186/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0390 - acc: 0.9868 - val_loss: 1.0166 - val_acc: 0.7911
Epoch 187/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0319 - acc: 0.9898 - val_loss: 1.0106 - val_acc: 0.7882
Epoch 188/300
546/546 [==================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



546/546 [==============================] - 19s 36ms/step - loss: 0.0253 - acc: 0.9920 - val_loss: 1.1827 - val_acc: 0.7680
Epoch 242/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0257 - acc: 0.9921 - val_loss: 1.1088 - val_acc: 0.7851
Epoch 243/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0250 - acc: 0.9912 - val_loss: 1.1495 - val_acc: 0.7742
Epoch 244/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0266 - acc: 0.9920 - val_loss: 1.2114 - val_acc: 0.7713
Epoch 245/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0235 - acc: 0.9924 - val_loss: 1.2157 - val_acc: 0.7731
Epoch 246/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0292 - acc: 0.9906 - val_loss: 1.1371 - val_acc: 0.7756
Epoch 247/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0273 - acc: 0.9910 - val_loss: 1.2345 - val_acc: 0.7566
Epoch 248/300
546/546 [==============================] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



546/546 [==============================] - 20s 36ms/step - loss: 0.0219 - acc: 0.9931 - val_loss: 1.1155 - val_acc: 0.7879
Epoch 276/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0217 - acc: 0.9934 - val_loss: 1.1498 - val_acc: 0.7841
Epoch 277/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0218 - acc: 0.9930 - val_loss: 1.1779 - val_acc: 0.7807
Epoch 278/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0241 - acc: 0.9921 - val_loss: 1.1920 - val_acc: 0.7786
Epoch 279/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0215 - acc: 0.9932 - val_loss: 1.1832 - val_acc: 0.7743
Epoch 280/300
546/546 [==============================] - 20s 36ms/step - loss: 0.0239 - acc: 0.9922 - val_loss: 1.1679 - val_acc: 0.7760
Epoch 281/300
546/546 [==============================] - 19s 36ms/step - loss: 0.0214 - acc: 0.9933 - val_loss: 1.1848 - val_acc: 0.7818
Epoch 282/300
546/546 [==============================] 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
exe_time

5872.486154556274

In [21]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea1_model.hdf5')
test_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv0_block0 (Conv2D)           (None, 32, 32, 64)   1792        input_11[0][0]                   
__________________________________________________________________________________________________
batchnorm0_block0 (BatchNormali (None, 32, 32, 64)   256         conv0_block0[0][0]               
__________________________________________________________________________________________________
relu0_block0 (Activation)       (None, 32, 32, 64)   0           batchnorm0_block0[0][0]          
__________________________________________________________________________________________________
conv1_bloc

In [22]:
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 83s 8ms/step


[1.0228634945854813, 0.7924]

batch_size = 32

In [ ]:
import pickle

epochs = 300

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea1_32_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea1_32_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

W0922 05:40:30.765764 139959264085760 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/300
1093/1093 [==============================] - 30s 27ms/step - loss: 1.6010 - acc: 0.4332 - val_loss: 2.0038 - val_acc: 0.3562
Epoch 2/300
1093/1093 [==============================] - 27s 25ms/step - loss: 1.3899 - acc: 0.5103 - val_loss: 1.4360 - val_acc: 0.4880
Epoch 3/300
1093/1093 [==============================] - 27s 25ms/step - loss: 1.3028 - acc: 0.5413 - val_loss: 1.5977 - val_acc: 0.4494
Epoch 4/300
1093/1093 [==============================] - 27s 25ms/step - loss: 1.2395 - acc: 0.5619 - val_loss: 1.4075 - val_acc: 0.5114
Epoch 5/300
1093/1093 [==============================] - 27s 25ms/step - loss: 1.1882 - acc: 0.5779 - val_loss: 1.3048 - val_acc: 0.5452
Epoch 6/300
1093/1093 [==============================] - 27s 25ms/step - loss: 1.1478 - acc: 0.5945 - val_loss: 1.4221 - val_acc: 0.5207
Epoch 7/300
1093/1093 [==============================] - 27s 25ms/step - loss: 1.1003 - acc: 0.6133 - val_loss: 1.2547 - val_acc: 0.5587
Epoch 8/300
1093/1093 [==================

1093/1093 [==============================] - 27s 25ms/step - loss: 0.0956 - acc: 0.9683 - val_loss: 0.8962 - val_acc: 0.7783
Epoch 120/300
1093/1093 [==============================] - 27s 25ms/step - loss: 0.0956 - acc: 0.9677 - val_loss: 0.8986 - val_acc: 0.7812
Epoch 121/300
1090/1093 [============================>.] - ETA: 0s - loss: 0.0946 - acc: 0.9677

In [12]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea1_32_model.hdf5')

score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)

10000/10000 [==============================] - 86s 9ms/step


In [13]:
score

[0.9464669375022655, 0.795]

## Experiment 2

Use the feature extractor **from every output of convolutional layers** of pretrain model to produce laten space, then concatenate with the original model
Here we use the VGG pretrain model from CIFAR 10 and the target task is CIFAR10 classification

In [14]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')
for layer in pretrain_vgg.layers:
    layer.trainable=False

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs)

vgg_outs = vgg(input_layer)

for layer in vgg_outs:
    vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_layer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)
vgg_gaps.append(skippedvgg_out)

concat_layer = concatenate(vgg_gaps)
outputs = Dense(10)(concat_layer)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

Create DensedVGG model with 3 blocks
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_3/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Concatenate  [<tf.Tensor 'cropping_block0_block2_3/strided_slice:0' shape=(?, 8, 8, 64) dtype=float32>, <tf.Tensor 'maxpooling_block1_3/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________

In [9]:
import pickle

epochs = 130

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea12_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea12_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

W0913 05:01:38.033998 140163647837952 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/130
546/546 [==============================] - 22s 41ms/step - loss: 1.4893 - acc: 0.4913 - val_loss: 1.8381 - val_acc: 0.3663
Epoch 2/130
546/546 [==============================] - 20s 37ms/step - loss: 1.2694 - acc: 0.5697 - val_loss: 1.5107 - val_acc: 0.4812
Epoch 3/130
546/546 [==============================] - 19s 35ms/step - loss: 1.1868 - acc: 0.5926 - val_loss: 1.3694 - val_acc: 0.5345
Epoch 4/130
546/546 [==============================] - 19s 36ms/step - loss: 1.1409 - acc: 0.6071 - val_loss: 1.2462 - val_acc: 0.5837
Epoch 5/130
546/546 [==============================] - 19s 36ms/step - loss: 1.0971 - acc: 0.6217 - val_loss: 1.2673 - val_acc: 0.5593
Epoch 6/130
546/546 [==============================] - 19s 36ms/step - loss: 1.0602 - acc: 0.6310 - val_loss: 1.5290 - val_acc: 0.4772
Epoch 7/130
546/546 [==============================] - 19s 36ms/step - loss: 1.0414 - acc: 0.6362 - val_loss: 1.2553 - val_acc: 0.5512
Epoch 8/130
546/546 [==============================] - 

546/546 [==============================] - 20s 36ms/step - loss: 0.0868 - acc: 0.9716 - val_loss: 1.1100 - val_acc: 0.7443
Epoch 122/130
546/546 [==============================] - 20s 36ms/step - loss: 0.0848 - acc: 0.9731 - val_loss: 1.2695 - val_acc: 0.7114
Epoch 123/130
546/546 [==============================] - 20s 36ms/step - loss: 0.0868 - acc: 0.9717 - val_loss: 1.1373 - val_acc: 0.7422
Epoch 124/130
546/546 [==============================] - 20s 36ms/step - loss: 0.0816 - acc: 0.9730 - val_loss: 1.1552 - val_acc: 0.7347
Epoch 125/130
546/546 [==============================] - 20s 36ms/step - loss: 0.0805 - acc: 0.9731 - val_loss: 1.3300 - val_acc: 0.7126
Epoch 126/130
546/546 [==============================] - 20s 36ms/step - loss: 0.0808 - acc: 0.9733 - val_loss: 1.1831 - val_acc: 0.7245
Epoch 127/130
546/546 [==============================] - 20s 36ms/step - loss: 0.0804 - acc: 0.9739 - val_loss: 1.1189 - val_acc: 0.7465
Epoch 128/130
546/546 [==============================] 

In [11]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea12_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 83s 8ms/step


[1.0389897839899291, 0.7529]

batch_size = 32

In [ ]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea12_32_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea12_32_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 29s 27ms/step - loss: 1.4874 - acc: 0.4915 - val_loss: 1.4351 - val_acc: 0.4944
Epoch 2/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.2891 - acc: 0.5540 - val_loss: 1.3448 - val_acc: 0.5311
Epoch 3/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.2154 - acc: 0.5770 - val_loss: 1.2646 - val_acc: 0.5587
Epoch 4/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.1785 - acc: 0.5865 - val_loss: 1.3030 - val_acc: 0.5452
Epoch 5/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.1447 - acc: 0.5985 - val_loss: 1.2361 - val_acc: 0.5644
Epoch 6/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.1160 - acc: 0.6080 - val_loss: 1.1761 - val_acc: 0.5963
Epoch 7/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.0897 - acc: 0.6181 - val_loss: 1.1731 - val_acc: 0.5909
Epoch 8/130
1093/1093 [==================

In [16]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea12_32_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 89s 9ms/step


[0.9581621086956785, 0.7734]

### Experiment 3

Compare between the VGG pretrained model with the current SkippedVGG

In [18]:
from keras.applications.vgg16 import VGG16

vgg = VGG16(include_top=False, input_shape=(32,32,3), weights='imagenet')

First, freeze all layers and train the defined top model

In [19]:
for layer in vgg.layers:
    layer.trainable = False
    
input_layer = Input((32,32,3))
# Build the model
vgg_out = vgg(input_layer)
# Top model
output_layer = GlobalAveragePooling2D()(vgg_out)
output_layer = Dense(10)(output_layer)
output_layer = BatchNormalization()(output_layer)
output_layer = Activation('softmax')(output_layer)

model3 = Model(inputs=input_layer, outputs=output_layer)
model3.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_11  (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                5130      
_________________________________________________________________
batch_normalization_4 (Batch (None, 10)                40        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
Total params: 14,719,858
Trainable params: 5,150
Non-trainable params: 14,714,708
____________________________________________________________

In [7]:
import pickle

epochs = 30

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea13_freezed_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model3.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea12_freezed_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

W0918 08:42:13.930425 139780496729856 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
546/546 [==============================] - 13s 23ms/step - loss: 1.8105 - acc: 0.3638 - val_loss: 1.7214 - val_acc: 0.3860
Epoch 2/30
546/546 [==============================] - 11s 21ms/step - loss: 1.6276 - acc: 0.4325 - val_loss: 1.6062 - val_acc: 0.4355
Epoch 3/30
546/546 [==============================] - 11s 20ms/step - loss: 1.5784 - acc: 0.4462 - val_loss: 1.5761 - val_acc: 0.4485
Epoch 4/30
546/546 [==============================] - 11s 20ms/step - loss: 1.5549 - acc: 0.4520 - val_loss: 1.5469 - val_acc: 0.4589
Epoch 5/30
546/546 [==============================] - 11s 20ms/step - loss: 1.5440 - acc: 0.4554 - val_loss: 1.5426 - val_acc: 0.4565
Epoch 6/30
546/546 [==============================] - 11s 20ms/step - loss: 1.5278 - acc: 0.4643 - val_loss: 1.5184 - val_acc: 0.4651
Epoch 7/30
546/546 [==============================] - 11s 20ms/step - loss: 1.5264 - acc: 0.4621 - val_loss: 1.5032 - val_acc: 0.4738
Epoch 8/30
546/546 [==============================] - 11s 20ms

In [10]:
model3 = load_model('/data/Quan/datasets/cifar10/transfer_idea13_freezed_model.hdf5')

In [20]:
for layer in model3.layers:
    layer.trainable=True
    
for layer in model3.layers[1].layers:
    layer.trainable=True

In [21]:
import pickle

epochs = 130

train_generator.reset()
test_generator.reset()
validation_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea13_finetuning_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model3.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea12_finetuning_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/130
546/546 [==============================] - 12s 21ms/step - loss: 1.4947 - acc: 0.4723 - val_loss: 1.4977 - val_acc: 0.4760
Epoch 2/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4949 - acc: 0.4706 - val_loss: 1.5224 - val_acc: 0.4657
Epoch 3/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4912 - acc: 0.4715 - val_loss: 1.4947 - val_acc: 0.4770
Epoch 4/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4927 - acc: 0.4725 - val_loss: 1.5160 - val_acc: 0.4649
Epoch 5/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4951 - acc: 0.4739 - val_loss: 1.5026 - val_acc: 0.4743
Epoch 6/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4912 - acc: 0.4737 - val_loss: 1.4806 - val_acc: 0.4825
Epoch 7/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4867 - acc: 0.4751 - val_loss: 1.5052 - val_acc: 0.4740
Epoch 8/130
546/546 [==============================] - 

546/546 [==============================] - 11s 20ms/step - loss: 1.4844 - acc: 0.4758 - val_loss: 1.4951 - val_acc: 0.4780
Epoch 122/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4871 - acc: 0.4724 - val_loss: 1.4898 - val_acc: 0.4809
Epoch 123/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4833 - acc: 0.4769 - val_loss: 1.4917 - val_acc: 0.4777
Epoch 124/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4864 - acc: 0.4762 - val_loss: 1.5040 - val_acc: 0.4736
Epoch 125/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4810 - acc: 0.4759 - val_loss: 1.4979 - val_acc: 0.4782
Epoch 126/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4852 - acc: 0.4758 - val_loss: 1.5022 - val_acc: 0.4750
Epoch 127/130
546/546 [==============================] - 11s 20ms/step - loss: 1.4794 - acc: 0.4767 - val_loss: 1.4897 - val_acc: 0.4787
Epoch 128/130
546/546 [==============================] 

In [17]:
test_generator.reset()
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea13_finetuning_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


10000/10000 [==============================] - 64s 6ms/step


[1.497175493601896, 0.4761]

batch_size = 32

In [20]:
import pickle

epochs = 30

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea13_32_freezed_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model3.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea13_32_freezed_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/30
1093/1093 [==============================] - 15s 14ms/step - loss: 1.7731 - acc: 0.3716 - val_loss: 1.6663 - val_acc: 0.4135
Epoch 2/30
1093/1093 [==============================] - 14s 13ms/step - loss: 1.6267 - acc: 0.4263 - val_loss: 1.5717 - val_acc: 0.4460
Epoch 3/30
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5850 - acc: 0.4412 - val_loss: 1.5471 - val_acc: 0.4560
Epoch 4/30
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5677 - acc: 0.4455 - val_loss: 1.5426 - val_acc: 0.4636
Epoch 5/30
1093/1093 [==============================] - 14s 13ms/step - loss: 1.5482 - acc: 0.4508 - val_loss: 1.5250 - val_acc: 0.4657
Epoch 6/30
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5498 - acc: 0.4506 - val_loss: 1.5122 - val_acc: 0.4675
Epoch 7/30
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5452 - acc: 0.4533 - val_loss: 1.5201 - val_acc: 0.4663
Epoch 8/30
1093/1093 [==========================

In [21]:
model3 = load_model('/data/Quan/datasets/cifar10/transfer_idea13_32_freezed_model.hdf5')

for layer in model3.layers:
    layer.trainable=True
    
for layer in model3.layers[1].layers:
    layer.trainable=True

In [23]:
import pickle

epochs = 130

train_generator.reset()
test_generator.reset()
validation_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea13_32_finetuning_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model3.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea12_32_finetuning_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

/usr/local/lib/python3.5/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/130
1093/1093 [==============================] - 15s 13ms/step - loss: 1.5222 - acc: 0.4599 - val_loss: 1.5082 - val_acc: 0.4741
Epoch 2/130
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5263 - acc: 0.4577 - val_loss: 1.5052 - val_acc: 0.4701
Epoch 3/130
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5202 - acc: 0.4614 - val_loss: 1.4996 - val_acc: 0.4733
Epoch 4/130
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5217 - acc: 0.4591 - val_loss: 1.5137 - val_acc: 0.4691
Epoch 5/130
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5205 - acc: 0.4601 - val_loss: 1.5042 - val_acc: 0.4740
Epoch 6/130
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5153 - acc: 0.4617 - val_loss: 1.4985 - val_acc: 0.4767
Epoch 7/130
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5153 - acc: 0.4628 - val_loss: 1.5056 - val_acc: 0.4707
Epoch 8/130
1093/1093 [==================

1093/1093 [==============================] - 14s 12ms/step - loss: 1.5002 - acc: 0.4716 - val_loss: 1.4836 - val_acc: 0.4828
Epoch 120/130
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5047 - acc: 0.4661 - val_loss: 1.4901 - val_acc: 0.4782
Epoch 121/130
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5038 - acc: 0.4689 - val_loss: 1.4972 - val_acc: 0.4765
Epoch 122/130
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5108 - acc: 0.4675 - val_loss: 1.4940 - val_acc: 0.4775
Epoch 123/130
1093/1093 [==============================] - 14s 13ms/step - loss: 1.5084 - acc: 0.4665 - val_loss: 1.4933 - val_acc: 0.4796
Epoch 124/130
1093/1093 [==============================] - 14s 12ms/step - loss: 1.5037 - acc: 0.4683 - val_loss: 1.5020 - val_acc: 0.4741
Epoch 125/130
1093/1093 [==============================] - 13s 12ms/step - loss: 1.5028 - acc: 0.4690 - val_loss: 1.4975 - val_acc: 0.4777
Epoch 126/130
1093/1093 [================

In [24]:
test_generator.reset()
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea13_32_finetuning_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

/usr/local/lib/python3.5/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


10000/10000 [==============================] - 69s 7ms/step


[1.4958488912238042, 0.4778]

## Experiment 4

Use extracted features from output of each conv block and feed to corresponding conv blocks of new model (concatenate with the current ones). The pre-train model can be trainable or non-trainable


In [5]:
def SkippedVGG_for_transfer(nb_blocks, input_layer, nb_layers, nb_neurons, extra_inputs, verbose=1):
    concats = []
    
    for i in range(nb_blocks):
        concats.append([])
        
    if type(nb_layers) is int:
        temp = nb_layers
        nb_layers = []
        for i in range(nb_blocks):
            nb_layers.append(temp)
    
    if verbose:
        print('Create DensedVGG model with ' + str(nb_blocks) + ' blocks, input layer = ', input_layer)
    
    for i in range(nb_blocks):
        if verbose:
            print('Create block ' + str(i) + ':')
        
        if i == 0: # First block
            b = input_layer
            inputs = b
        
        # check all layers before
        if len(concats[i-1]) > 1:
            if (i-1) < len(extra_inputs):
                print('Add extra inputs ', extra_inputs[i-1])
                temp_concats = concats[i-1]
                temp_concats.append(extra_inputs[i-1])
                b = concatenate(temp_concats)
            else:   
                print('No extra inputs at block ', i)
                b = concatenate(concats[i-1])
        elif len(concats[i-1]) == 1:
            if verbose:
                print('Get direct output from the previous block ', concats[i-1])
#             b = concats[i-1][0]
            if (i-1) < len(extra_inputs):
                print('Add extra inputs ', extra_inputs[i-1])
                temp_concats = concats[i-1]
                temp_concats.append(extra_inputs[i-1])
                b = concatenate(temp_concats)
            else:   
                print('No extra inputs at block ', i)
                b = concatenate(concats[i-1][0])
            
        # create main block
        b = define_block(b, nb_layers[i], nb_neurons[i], index=i)
        concats[i].append(b)
        
        
        # create cropping layer
        for j in range(i+1, nb_blocks):
            if verbose:
                print('-- create skipped connection from block '+ str(i) + ' to block ' + str(j+1) + ' ...')
                
            src_shape = b.get_shape()
            src_shape = (int(src_shape[1]), int(src_shape[2]), int(src_shape[3]))
            dst_shape = (src_shape[0]//2**(j-i), src_shape[1]//2**(j-i), src_shape[2])
            
            print(src_shape, dst_shape)
            h = src_shape[0] - dst_shape[0]
            w = src_shape[1] - dst_shape[1]
            if h % 2 == 0:
                h_0 = h // 2
                h_1 = h // 2
            else:
                h_0 = h // 2
                h_1 = (h // 2) + 1
            
            if w % 2 == 0:
                w_0 = w // 2
                w_1 = w // 2
            else:
                w_0 = w // 2
                w_1 = (w // 2) + 1
                
            concat_layer = Cropping2D(((h_0,h_1),(w_0,w_1)), name='cropping_block' + str(i) + '_block' + str(j+1))(b)
            concats[j].append(concat_layer)
            
    if len(concats[nb_blocks-1]) == 1:
        outputs = concats[nb_blocks-1]
    else:
        outputs = concatenate(concats[nb_blocks-1])
        
    return outputs

In [64]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:2])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_28/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_28/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_44:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_19/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_28/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Add extra inputs  Tensor("model_28/block2_pool/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
__________________________________________________________________________________________________
Layer (type)    

In [ ]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea14_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea14_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 32s 29ms/step - loss: 1.5928 - acc: 0.4375 - val_loss: 1.7211 - val_acc: 0.3948
Epoch 2/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.3523 - acc: 0.5264 - val_loss: 1.6106 - val_acc: 0.4675
Epoch 3/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.2308 - acc: 0.5701 - val_loss: 1.2443 - val_acc: 0.5704
Epoch 4/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.1528 - acc: 0.5985 - val_loss: 1.2667 - val_acc: 0.5607
Epoch 5/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.0872 - acc: 0.6218 - val_loss: 1.3130 - val_acc: 0.5583
Epoch 6/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.0274 - acc: 0.6395 - val_loss: 1.1442 - val_acc: 0.6069
Epoch 7/130
1093/1093 [==============================] - 28s 25ms/step - loss: 0.9757 - acc: 0.6605 - val_loss: 1.2506 - val_acc: 0.5706
Epoch 8/130
1093/1093 [==================

In [66]:
print('exe time: ', exe_time)

exe time:  3568.3500208854675


In [68]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea14_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 72s 7ms/step


[0.698373294539816, 0.8193]

### Experiment 4.2
Using more than 3 blocks

In [81]:
from keras.applications.vgg16 import VGG16
del(model)
input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:4])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=5, input_layer=input_layer, 
                            nb_layers=[2,2,3,3,3], nb_neurons=[64,128,512,512,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_42/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_42/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>, <tf.Tensor 'model_42/block3_pool/MaxPool:0' shape=(?, 4, 4, 256) dtype=float32>, <tf.Tensor 'model_42/block4_pool/MaxPool:0' shape=(?, 2, 2, 512) dtype=float32>]
Create DensedVGG model with 5 blocks, input layer =  Tensor("input_64:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
-- create skipped connection from block 0 to block 4 ...
(16, 16, 64) (2, 2, 64)
-- create skipped connection from block 0 to block 5 ...
(16, 16, 64) (1, 1, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_32/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_42/block1_pool/MaxPool:0", shape=(?, 16, 16, 64),

In [ ]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea14.2_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea14.2_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 87s 80ms/step - loss: 1.6047 - acc: 0.4306 - val_loss: 2.1076 - val_acc: 0.3281
Epoch 2/130
1093/1093 [==============================] - 80s 73ms/step - loss: 1.3459 - acc: 0.5239 - val_loss: 1.4224 - val_acc: 0.5111
Epoch 3/130
1093/1093 [==============================] - 80s 73ms/step - loss: 1.2328 - acc: 0.5678 - val_loss: 1.2323 - val_acc: 0.5680
Epoch 4/130
1093/1093 [==============================] - 80s 73ms/step - loss: 1.1512 - acc: 0.5996 - val_loss: 1.4612 - val_acc: 0.5098
Epoch 5/130
1093/1093 [==============================] - 80s 73ms/step - loss: 1.0822 - acc: 0.6235 - val_loss: 1.1208 - val_acc: 0.6140
Epoch 6/130
1093/1093 [==============================] - 80s 73ms/step - loss: 1.0269 - acc: 0.6435 - val_loss: 1.1049 - val_acc: 0.6158
Epoch 7/130
1093/1093 [==============================] - 80s 73ms/step - loss: 0.9726 - acc: 0.6630 - val_loss: 1.4017 - val_acc: 0.5520
Epoch 8/130
1093/1093 [==================

In [83]:
exe_time

10431.521920204163

In [84]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea14.2_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 122s 12ms/step


[0.877768070437959, 0.7949]

### Experiment 4.3

freeze all layers

In [86]:
from keras.applications.vgg16 import VGG16
del(model)
input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

for layer in pretrain_vgg.layers:
    layer.trainable = False

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:2])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_46/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_46/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_68:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_35/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_46/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Add extra inputs  Tensor("model_46/block2_pool/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
__________________________________________________________________________________________________
Layer (type)    

In [ ]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea14.3_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea14.3_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 34s 31ms/step - loss: 1.5558 - acc: 0.4526 - val_loss: 1.6641 - val_acc: 0.4422
Epoch 2/130
1093/1093 [==============================] - 27s 24ms/step - loss: 1.3389 - acc: 0.5320 - val_loss: 1.3510 - val_acc: 0.5196
Epoch 3/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.2315 - acc: 0.5678 - val_loss: 1.4610 - val_acc: 0.4868
Epoch 4/130
1093/1093 [==============================] - 27s 24ms/step - loss: 1.1466 - acc: 0.5950 - val_loss: 1.3095 - val_acc: 0.5605
Epoch 5/130
1093/1093 [==============================] - 27s 24ms/step - loss: 1.0895 - acc: 0.6165 - val_loss: 1.0113 - val_acc: 0.6426
Epoch 6/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.0395 - acc: 0.6337 - val_loss: 1.1144 - val_acc: 0.6211
Epoch 7/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.0003 - acc: 0.6490 - val_loss: 1.0834 - val_acc: 0.6269
Epoch 8/130
1093/1093 [==================

In [88]:
exe_time

3528.7938346862793

In [89]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea14.3_model.hdf5')
test_generator.reset()
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 50s 5ms/step


[0.9533610496851498, 0.7807]

### Experiment 4.4
freeze all layers and use 5 blocks

In [90]:
from keras.applications.vgg16 import VGG16
del(model)
input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

for layer in pretrain_vgg.layers:
    layer.trainable = False

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:4])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=5, input_layer=input_layer, 
                            nb_layers=[2,2,3,3,3], nb_neurons=[64,128,512,512,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_48/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_48/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>, <tf.Tensor 'model_48/block3_pool/MaxPool:0' shape=(?, 4, 4, 256) dtype=float32>, <tf.Tensor 'model_48/block4_pool/MaxPool:0' shape=(?, 2, 2, 512) dtype=float32>]
Create DensedVGG model with 5 blocks, input layer =  Tensor("input_70:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
-- create skipped connection from block 0 to block 4 ...
(16, 16, 64) (2, 2, 64)
-- create skipped connection from block 0 to block 5 ...
(16, 16, 64) (1, 1, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_37/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_48/block1_pool/MaxPool:0", shape=(?, 16, 16, 64),

In [ ]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea14.4_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea14.4_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 77s 71ms/step - loss: 1.4318 - acc: 0.5020 - val_loss: 1.4810 - val_acc: 0.4943
Epoch 2/130
1093/1093 [==============================] - 68s 63ms/step - loss: 1.2335 - acc: 0.5709 - val_loss: 1.2646 - val_acc: 0.5509
Epoch 3/130
1093/1093 [==============================] - 69s 63ms/step - loss: 1.1517 - acc: 0.5982 - val_loss: 1.2733 - val_acc: 0.5616
Epoch 4/130
1093/1093 [==============================] - 68s 63ms/step - loss: 1.0897 - acc: 0.6206 - val_loss: 1.1574 - val_acc: 0.5985
Epoch 5/130
1093/1093 [==============================] - 69s 63ms/step - loss: 1.0366 - acc: 0.6370 - val_loss: 0.9580 - val_acc: 0.6669
Epoch 6/130
1093/1093 [==============================] - 69s 63ms/step - loss: 1.0068 - acc: 0.6502 - val_loss: 1.0506 - val_acc: 0.6315
Epoch 7/130
1093/1093 [==============================] - 68s 63ms/step - loss: 0.9546 - acc: 0.6685 - val_loss: 0.9184 - val_acc: 0.6793
Epoch 8/130
1093/1093 [==================

In [92]:
exe_time

8984.064810276031

In [93]:
del(test_model)

test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea14.4_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 115s 11ms/step


[1.0620492207547167, 0.7787]

### Experiment 4.5
same as 4 but use smaller learning rate

In [30]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

del(model)
pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:2])
vgg.compile(loss=categorical_crossentropy, optimizer=Adam(1e-5), metrics=['accuracy'])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(1e-4), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_21/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_21/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_21:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_10/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_21/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Add extra inputs  Tensor("model_21/block2_pool/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
__________________________________________________________________________________________________
Layer (type)    

In [31]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea14.5_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea14.5_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 32s 29ms/step - loss: 1.5550 - acc: 0.4655 - val_loss: 1.6195 - val_acc: 0.4173
Epoch 2/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.3462 - acc: 0.5547 - val_loss: 1.4837 - val_acc: 0.4989
Epoch 3/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.2445 - acc: 0.5931 - val_loss: 1.1981 - val_acc: 0.6013
Epoch 4/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.1599 - acc: 0.6274 - val_loss: 1.3115 - val_acc: 0.5777
Epoch 5/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.0944 - acc: 0.6498 - val_loss: 1.2138 - val_acc: 0.5850
Epoch 6/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.0471 - acc: 0.6626 - val_loss: 1.0528 - val_acc: 0.6549
Epoch 7/130
1093/1093 [==============================] - 27s 25ms/step - loss: 1.0096 - acc: 0.6757 - val_loss: 0.9619 - val_acc: 0.6927
Epoch 8/130
1093/1093 [==================

1093/1093 [==============================] - 28s 25ms/step - loss: 0.1071 - acc: 0.9699 - val_loss: 0.6841 - val_acc: 0.8040
Epoch 120/130
1093/1093 [==============================] - 27s 25ms/step - loss: 0.1015 - acc: 0.9717 - val_loss: 0.6652 - val_acc: 0.8135
Epoch 121/130
1093/1093 [==============================] - 27s 25ms/step - loss: 0.1015 - acc: 0.9721 - val_loss: 0.6523 - val_acc: 0.8093
Epoch 122/130
1093/1093 [==============================] - 27s 25ms/step - loss: 0.1016 - acc: 0.9719 - val_loss: 0.6513 - val_acc: 0.8139
Epoch 123/130
1093/1093 [==============================] - 27s 25ms/step - loss: 0.0999 - acc: 0.9715 - val_loss: 0.7009 - val_acc: 0.8064
Epoch 124/130
1093/1093 [==============================] - 27s 25ms/step - loss: 0.0990 - acc: 0.9728 - val_loss: 0.6925 - val_acc: 0.8067
Epoch 125/130
1093/1093 [==============================] - 28s 25ms/step - loss: 0.0974 - acc: 0.9735 - val_loss: 0.6855 - val_acc: 0.8059
Epoch 126/130
1093/1093 [================

In [32]:
del(test_model)
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea14.5_model.hdf5')
test_generator.reset()
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 46s 5ms/step


[0.6463626317824214, 0.8109]

In [ ]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:2])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

### Experiment 5

Same as **Experiment 4** but keep all rest of conv blocks and apply GAP, then concatenate with the output of new model

In [75]:
from keras.applications.vgg16 import VGG16
del(model)

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs)

vgg_outs = vgg(input_layer)
print(vgg_outs)

for layer in vgg_outs[2:]:
    vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs[:2])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)
vgg_gaps.append(skippedvgg_out)
outputs = concatenate(vgg_gaps)

outputs = Dense(10)(outputs)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_37/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_37/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>, <tf.Tensor 'model_37/block3_pool/MaxPool:0' shape=(?, 4, 4, 256) dtype=float32>, <tf.Tensor 'model_37/block4_pool/MaxPool:0' shape=(?, 2, 2, 512) dtype=float32>, <tf.Tensor 'model_37/block5_pool/MaxPool:0' shape=(?, 1, 1, 512) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_58:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_28/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_37/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 1

In [ ]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea15_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea15_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 58s 53ms/step - loss: 1.8820 - acc: 0.2761 - val_loss: 1.7512 - val_acc: 0.3214
Epoch 2/130
1093/1093 [==============================] - 52s 47ms/step - loss: 1.6437 - acc: 0.3900 - val_loss: 2.0209 - val_acc: 0.3237
Epoch 3/130
1093/1093 [==============================] - 51s 47ms/step - loss: 1.4718 - acc: 0.4592 - val_loss: 1.7065 - val_acc: 0.3774
Epoch 4/130
1093/1093 [==============================] - 51s 47ms/step - loss: 1.3668 - acc: 0.4988 - val_loss: 1.6042 - val_acc: 0.4634
Epoch 5/130
1093/1093 [==============================] - 52s 48ms/step - loss: 1.2831 - acc: 0.5411 - val_loss: 1.2207 - val_acc: 0.5541
Epoch 6/130
1093/1093 [==============================] - 52s 47ms/step - loss: 1.2132 - acc: 0.5665 - val_loss: 1.2823 - val_acc: 0.5635
Epoch 7/130
1093/1093 [==============================] - 51s 47ms/step - loss: 1.1627 - acc: 0.5874 - val_loss: 1.1595 - val_acc: 0.5919
Epoch 8/130
1093/1093 [==================

In [77]:
print('exe time: ', exe_time)

exe time:  6736.114166021347


In [78]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea15_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 72s 7ms/step


[0.765161869376994, 0.7932]

### Experiment 5.2



In [6]:
from keras.applications.vgg16 import VGG16
# del(model)

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

for layer in pretrain_vgg.layers[7:]:
    layer.trainable = False

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs)

vgg_outs = vgg(input_layer)
print(vgg_outs)

for layer in vgg_outs[2:]:
    vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs[:2])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)
vgg_gaps.append(skippedvgg_out)
outputs = concatenate(vgg_gaps)

outputs = Dense(10)(outputs)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

W1002 05:28:10.060826 140535202330368 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1002 05:28:10.061772 140535202330368 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1002 05:28:10.064476 140535202330368 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1002 05:28:10.083518 140535202330368 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1002 05:28:10.304884 140535202330368 deprecation_wrapp

[<tf.Tensor 'model_1/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_1/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>, <tf.Tensor 'model_1/block3_pool/MaxPool:0' shape=(?, 4, 4, 256) dtype=float32>, <tf.Tensor 'model_1/block4_pool/MaxPool:0' shape=(?, 2, 2, 512) dtype=float32>, <tf.Tensor 'model_1/block5_pool/MaxPool:0' shape=(?, 1, 1, 512) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_1:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_1/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4,

W1002 05:28:11.180184 140535202330368 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv0_block0 (Conv2D)           (None, 32, 32, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batchnorm0_block0 (BatchNormali (None, 32, 32, 64)   256         conv0_block0[0][0]               
__________________________________________________________________________________________________
relu0_block0 (Activation)       (None, 32, 32, 64)   0           batchnorm0_block0[0][0]          
__________________________________________________________________________________________________
conv1_bloc

In [7]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea15.2_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea15.2_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

W1002 05:28:18.475156 140535202330368 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/130
1093/1093 [==============================] - 38s 35ms/step - loss: 1.6226 - acc: 0.4232 - val_loss: 1.6144 - val_acc: 0.4272
Epoch 2/130
1093/1093 [==============================] - 36s 33ms/step - loss: 1.2908 - acc: 0.5544 - val_loss: 1.3326 - val_acc: 0.5198
Epoch 3/130
1093/1093 [==============================] - 36s 33ms/step - loss: 1.1636 - acc: 0.5964 - val_loss: 1.2274 - val_acc: 0.5831
Epoch 4/130
1093/1093 [==============================] - 36s 33ms/step - loss: 1.0895 - acc: 0.6207 - val_loss: 1.1582 - val_acc: 0.5983
Epoch 5/130
1093/1093 [==============================] - 36s 33ms/step - loss: 1.0346 - acc: 0.6377 - val_loss: 1.3172 - val_acc: 0.5622
Epoch 6/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.9928 - acc: 0.6539 - val_loss: 1.0620 - val_acc: 0.6283
Epoch 7/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.9546 - acc: 0.6649 - val_loss: 1.0134 - val_acc: 0.6507
Epoch 8/130
1093/1093 [==================

1093/1093 [==============================] - 36s 33ms/step - loss: 0.1117 - acc: 0.9619 - val_loss: 0.7306 - val_acc: 0.8067
Epoch 120/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.1136 - acc: 0.9605 - val_loss: 0.8670 - val_acc: 0.7881
Epoch 121/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.1129 - acc: 0.9632 - val_loss: 0.7665 - val_acc: 0.8015
Epoch 122/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.1097 - acc: 0.9626 - val_loss: 0.8236 - val_acc: 0.7861
Epoch 123/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.1054 - acc: 0.9646 - val_loss: 0.9060 - val_acc: 0.7793
Epoch 124/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.1073 - acc: 0.9639 - val_loss: 0.7734 - val_acc: 0.8027
Epoch 125/130
1093/1093 [==============================] - 36s 33ms/step - loss: 0.1047 - acc: 0.9660 - val_loss: 0.8193 - val_acc: 0.7950
Epoch 126/130
1093/1093 [================

In [8]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea15.2_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 65s 7ms/step


[0.7668102605718647, 0.7975]

### Experiment 6

In [18]:
from keras.applications.vgg16 import VGG16
del(model)

input_layer = Input((32,32,3))
conv_indices=[3,6,18]
# vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

for layer in pretrain_vgg.layers[7:]:
    layer.trainable = False

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs)

vgg_outs = vgg(input_layer)

# top model for pretrain_vgg
vgg_top = Flatten()(vgg_outs[-1])
vgg_top = Dense(10)(vgg_top)
vgg_top = BatchNormalization()(vgg_top)
vgg_top = Activation('softmax')(vgg_top)


print(vgg_outs)

# for layer in vgg_outs[2:]:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs[:2])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

# Ensamble
final_output = concatenate([vgg_top, outputs])
final_output = Dense(10)(final_output)
final_output = BatchNormalization()(final_output)
final_output = Activation('softmax')(final_output)

model = Model(inputs=input_layer, outputs=final_output)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_13/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_13/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>, <tf.Tensor 'model_13/block5_pool/MaxPool:0' shape=(?, 1, 1, 512) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_13:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_7/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_13/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Add extra inputs  Tensor("model_13/block2_pool/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
___________________________________

In [15]:
vgg_outs[-1]

<tf.Tensor 'model_9/block5_pool/MaxPool:0' shape=(?, 1, 1, 512) dtype=float32>

In [19]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea16_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea16_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 44s 40ms/step - loss: 1.7231 - acc: 0.3848 - val_loss: 1.9341 - val_acc: 0.3413
Epoch 2/130
1093/1093 [==============================] - 41s 38ms/step - loss: 1.4676 - acc: 0.4820 - val_loss: 1.5523 - val_acc: 0.4667
Epoch 3/130
1093/1093 [==============================] - 41s 37ms/step - loss: 1.3469 - acc: 0.5242 - val_loss: 1.3226 - val_acc: 0.5408
Epoch 4/130
1093/1093 [==============================] - 41s 37ms/step - loss: 1.2619 - acc: 0.5555 - val_loss: 1.2820 - val_acc: 0.5438
Epoch 5/130
1093/1093 [==============================] - 41s 37ms/step - loss: 1.2003 - acc: 0.5794 - val_loss: 1.2585 - val_acc: 0.5553
Epoch 6/130
1093/1093 [==============================] - 41s 38ms/step - loss: 1.1412 - acc: 0.5983 - val_loss: 1.3935 - val_acc: 0.5132
Epoch 7/130
1093/1093 [==============================] - 41s 38ms/step - loss: 1.0990 - acc: 0.6163 - val_loss: 1.5525 - val_acc: 0.4840
Epoch 8/130
1093/1093 [==================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1093/1093 [==============================] - 41s 37ms/step - loss: 0.1032 - acc: 0.9644 - val_loss: 0.9380 - val_acc: 0.7843
Epoch 127/130
1093/1093 [==============================] - 41s 37ms/step - loss: 0.0975 - acc: 0.9671 - val_loss: 0.8269 - val_acc: 0.7922
Epoch 128/130
1093/1093 [==============================] - 41s 37ms/step - loss: 0.0987 - acc: 0.9664 - val_loss: 0.8456 - val_acc: 0.7935
Epoch 129/130
1093/1093 [==============================] - 41s 38ms/step - loss: 0.0982 - acc: 0.9686 - val_loss: 0.7546 - val_acc: 0.8123
Epoch 130/130
1093/1093 [==============================] - 41s 37ms/step - loss: 0.0945 - acc: 0.9688 - val_loss: 0.9988 - val_acc: 0.7662
exe time:  5365.481000185013


In [22]:
# del(test_model)
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea16_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 99s 10ms/step


[0.7575436492246583, 0.8124]

### Experiment 7

In [25]:
from keras.applications.vgg16 import VGG16
del(model)

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
# vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

for layer in pretrain_vgg.layers:
    layer.trainable = False

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs)

vgg_outs = vgg(input_layer)

concat_outputs = []

for output_layer in vgg_outs:
    partial_output = GlobalAveragePooling2D()(output_layer)
    partial_output = Dense(10)(partial_output)
    partial_output = BatchNormalization()(partial_output)
    partial_output = Activation('softmax')(partial_output)
    concat_outputs.append(partial_output)
    
final_output = concatenate(concat_outputs)
final_output = Dense(10)(final_output)
final_output = BatchNormalization()(final_output)
final_output = Activation('softmax')(final_output)

model = Model(inputs=input_layer, outputs=final_output)
model.compile(loss=categorical_crossentropy, optimizer=Adam(1e-4), metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
model_17 (Model)                multiple             14714688    input_17[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_19 (Gl (None, 64)           0           model_17[1][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_20 (Gl (None, 128)          0           model_17[1][1]                   
__________________________________________________________________________________________________
global_ave

In [26]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea17_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea17_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

Epoch 1/130
1093/1093 [==============================] - 19s 17ms/step - loss: 2.0892 - acc: 0.2649 - val_loss: 1.8107 - val_acc: 0.3631
Epoch 2/130
1093/1093 [==============================] - 15s 14ms/step - loss: 1.7381 - acc: 0.3932 - val_loss: 1.6501 - val_acc: 0.4342
Epoch 3/130
1093/1093 [==============================] - 15s 14ms/step - loss: 1.6406 - acc: 0.4336 - val_loss: 1.5801 - val_acc: 0.4621
Epoch 4/130
1093/1093 [==============================] - 14s 13ms/step - loss: 1.5766 - acc: 0.4574 - val_loss: 1.5360 - val_acc: 0.4733
Epoch 5/130
1093/1093 [==============================] - 15s 13ms/step - loss: 1.5296 - acc: 0.4762 - val_loss: 1.4981 - val_acc: 0.4848
Epoch 6/130
1093/1093 [==============================] - 15s 13ms/step - loss: 1.4924 - acc: 0.4912 - val_loss: 1.4712 - val_acc: 0.4969
Epoch 7/130
1093/1093 [==============================] - 14s 13ms/step - loss: 1.4630 - acc: 0.4959 - val_loss: 1.4163 - val_acc: 0.5172
Epoch 8/130
1093/1093 [==================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1093/1093 [==============================] - 15s 14ms/step - loss: 1.0996 - acc: 0.6114 - val_loss: 1.1302 - val_acc: 0.6093
Epoch 115/130
1093/1093 [==============================] - 15s 14ms/step - loss: 1.0989 - acc: 0.6127 - val_loss: 1.1123 - val_acc: 0.6131
Epoch 116/130
1093/1093 [==============================] - 15s 14ms/step - loss: 1.0971 - acc: 0.6105 - val_loss: 1.1061 - val_acc: 0.6177
Epoch 117/130
1093/1093 [==============================] - 15s 13ms/step - loss: 1.0971 - acc: 0.6130 - val_loss: 1.0900 - val_acc: 0.6203
Epoch 118/130
1093/1093 [==============================] - 15s 13ms/step - loss: 1.1053 - acc: 0.6088 - val_loss: 1.1097 - val_acc: 0.6116
Epoch 119/130
1093/1093 [==============================] - 15s 13ms/step - loss: 1.0925 - acc: 0.6157 - val_loss: 1.1436 - val_acc: 0.6037
Epoch 120/130
1093/1093 [==============================] - 15s 14ms/step - loss: 1.0983 - acc: 0.6108 - val_loss: 1.1228 - val_acc: 0.6106
Epoch 121/130
1093/1093 [================

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
del(test_model)
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea17_model.hdf5')
test_generator.reset()

score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 91s 9ms/step


[1.0767300821033248, 0.6258]

In [ ]:
-------------------------------------------------------------------------------------

In [9]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:2])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_3/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_3/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_3:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_2/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_3/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Add extra inputs  Tensor("model_3/block2_pool/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
__________________________________________________________________________________________________
Layer (type)          

In [105]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)
    
vgg_input, vgg_outputs = vgg_extend(input_layer, weights=[k.get_weights() for k in pretrain_vgg.layers], vgg_conv

input_74 (None, None, None, 3)
block1_conv1 (None, None, None, 64)
block1_conv2 (None, None, None, 64)
block1_pool (None, None, None, 64)
block2_conv1 (None, None, None, 128)
block2_conv2 (None, None, None, 128)
block2_pool (None, None, None, 128)
block3_conv1 (None, None, None, 256)
block3_conv2 (None, None, None, 256)
block3_conv3 (None, None, None, 256)
block3_pool (None, None, None, 256)
block4_conv1 (None, None, None, 512)
block4_conv2 (None, None, None, 512)
block4_conv3 (None, None, None, 512)
block4_pool (None, None, None, 512)
block5_conv1 (None, None, None, 512)
block5_conv2 (None, None, None, 512)
block5_conv3 (None, None, None, 512)
block5_pool (None, None, None, 512)


In [108]:
b = pretrain_vgg.layers[3]
# b = Conv2D(128,(3,3), padding='same')(b)
b.get_weights()

[]

In [ ]:
def vgg_extend(input_layer, weights, num_blocks, extend_inputs):
    neurons = [64,128,256,512]
    counter = 1
    b = input_layer
    inputs = b
    outputs = []
    for block in num_blocks:
        b =  Conv2D(neurons[block], kernel_size=(3,3), strides=1, padding='same', name='conv' + str(i) + '_block' + str(block))(b)
        b.set_weights(weights[counter])
        counter += 1
        
        b =  Conv2D(neurons[block], kernel_size=(3,3), strides=1, padding='same', name='conv' + str(i) + '_block' + str(block))(b)
        b.set_weights(weights[counter])
        counter += 1
        
        b = MaxPooling2D(pool_size=(2,2), name='maxpooling_block' + str(block))(b)
        b.set_weights(weights[counter])
        counter += 1
        
        outputs.append(b)
        if exist(extend_inputs[block-1]):
            b = concatenate([extend_inputs[block-1], b])
        else:
            print('Not exist input at ', block-1)
        
        
    return inputs, outputs 
        

In [113]:
pretrain_layers = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')
for layer in pretrain_vgg.layers[7:]:
    print(layer.name)

block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


## Experiment 8

In [5]:


def SkippedVGG_for_transfer_2(nb_blocks, input_layer, nb_layers, nb_neurons, extra_inputs, verbose=1):
    concats = []
    
    for i in range(nb_blocks):
        concats.append([])
        
    if type(nb_layers) is int:
        temp = nb_layers
        nb_layers = []
        for i in range(nb_blocks):
            nb_layers.append(temp)
    
    if verbose:
        print('Create DensedVGG model with ' + str(nb_blocks) + ' blocks, input layer = ', input_layer)
    
    for i in range(nb_blocks):
        if verbose:
            print('Create block ' + str(i) + ':')
        
        if i == 0: # First block
            b = input_layer
            inputs = b
        
        # check all layers before
        if len(concats[i-1]) > 1:
            if (i-1) < len(extra_inputs):
                print('Add extra inputs ', extra_inputs[i-1])
                temp_concats = concats[i-1]
                temp_concats.append(extra_inputs[i-1])
                b = concatenate(temp_concats)
                
                # Go through 1x1 Conv2D
                b = Conv2D(nb_neurons[i], (1,1), strides=1, padding='same')(b)
                
            else:   
                print('No extra inputs at block ', i)
                b = concatenate(concats[i-1])
        elif len(concats[i-1]) == 1:
            if verbose:
                print('Get direct output from the previous block ', concats[i-1])
#             b = concats[i-1][0]
            if (i-1) < len(extra_inputs):
                print('Add extra inputs ', extra_inputs[i-1])
                temp_concats = concats[i-1]
                temp_concats.append(extra_inputs[i-1])
                b = concatenate(temp_concats)
                b = Conv2D(nb_neurons[i], (1,1), strides=1, padding='same')(b)
                
            else:   
                print('No extra inputs at block ', i)
                b = concatenate(concats[i-1][0])
            
        # create main block
        b = define_block(b, nb_layers[i], nb_neurons[i], index=i)
        concats[i].append(b)
        
        
        # create cropping layer
        for j in range(i+1, nb_blocks):
            if verbose:
                print('-- create skipped connection from block '+ str(i) + ' to block ' + str(j+1) + ' ...')
                
            src_shape = b.get_shape()
            src_shape = (int(src_shape[1]), int(src_shape[2]), int(src_shape[3]))
            dst_shape = (src_shape[0]//2**(j-i), src_shape[1]//2**(j-i), src_shape[2])
            
            print(src_shape, dst_shape)
            h = src_shape[0] - dst_shape[0]
            w = src_shape[1] - dst_shape[1]
            if h % 2 == 0:
                h_0 = h // 2
                h_1 = h // 2
            else:
                h_0 = h // 2
                h_1 = (h // 2) + 1
            
            if w % 2 == 0:
                w_0 = w // 2
                w_1 = w // 2
            else:
                w_0 = w // 2
                w_1 = (w // 2) + 1
                
            concat_layer = Cropping2D(((h_0,h_1),(w_0,w_1)), name='cropping_block' + str(i) + '_block' + str(j+1))(b)
            concats[j].append(concat_layer)
            
    if len(concats[nb_blocks-1]) == 1:
        outputs = concats[nb_blocks-1]
    else:
        outputs = concatenate(concats[nb_blocks-1])
        
    return outputs


In [7]:
from keras.applications.vgg16 import VGG16

input_layer = Input((32,32,3))
conv_indices=[3,6,10,14,18]
vgg_gaps = []

pretrain_vgg = VGG16(include_top=False, weights='imagenet')

vgg_conv_outputs = []
for i in conv_indices:
    vgg_conv_outputs.append(pretrain_vgg.layers[i].output)

vgg = Model(inputs=pretrain_vgg.input, outputs=vgg_conv_outputs[:2])

vgg_outs = vgg(input_layer)
print(vgg_outs)

# for layer in vgg_outs:
#     vgg_gaps.append(GlobalAveragePooling2D()(layer))

skippedvgg_out = SkippedVGG_for_transfer_2(nb_blocks=3, input_layer=input_layer, 
                            nb_layers=[2,2,3], nb_neurons=[64,128,512], extra_inputs=vgg_outs)

# vgg_concats = GlobalAveragePooling2D()(vgg_outs[3:])

skippedvgg_out = GlobalAveragePooling2D()(skippedvgg_out)

# outputs = concatenate([vgg_concats, skippedvgg_out])
outputs = Dense(10)(skippedvgg_out)
outputs = BatchNormalization()(outputs)
outputs = Activation('softmax')(outputs)

model = Model(inputs=input_layer, outputs=outputs)
model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
model.summary()

[<tf.Tensor 'model_3/block1_pool/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>, <tf.Tensor 'model_3/block2_pool/MaxPool:0' shape=(?, 8, 8, 128) dtype=float32>]
Create DensedVGG model with 3 blocks, input layer =  Tensor("input_3:0", shape=(?, 32, 32, 3), dtype=float32)
Create block 0:
-- create skipped connection from block 0 to block 2 ...
(16, 16, 64) (8, 8, 64)
-- create skipped connection from block 0 to block 3 ...
(16, 16, 64) (4, 4, 64)
Create block 1:
Get direct output from the previous block  [<tf.Tensor 'maxpooling_block0_1/MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>]
Add extra inputs  Tensor("model_3/block1_pool/MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)
-- create skipped connection from block 1 to block 3 ...
(8, 8, 128) (4, 4, 128)
Create block 2:
Add extra inputs  Tensor("model_3/block2_pool/MaxPool:0", shape=(?, 8, 8, 128), dtype=float32)
__________________________________________________________________________________________________
Layer (type)          

In [10]:
import pickle

epochs = 130

train_generator.reset()
validation_generator.reset()
test_generator.reset()

mcp = ModelCheckpoint('/data/Quan/datasets/cifar10/transfer_idea18_model.hdf5', monitor='val_acc', 
                      save_best_only=True, save_weights_only=False)

start = time.time()
history = model.fit_generator(train_generator, steps_per_epoch=len(train_images)//batch_size, 
                            epochs=epochs, verbose=1, validation_data=validation_generator, 
                              validation_steps=len(validation_images)//batch_size, callbacks=[mcp])

exe_time = time.time() - start

print('exe time: ', exe_time)
with open('/data/Quan/datasets/cifar10/transfer_idea18_history.hdf5', 'wb') as dt:
    pickle.dump(history.history, dt)

W1016 09:54:47.163458 140057288775424 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/130
1093/1093 [==============================] - 32s 29ms/step - loss: 1.6365 - acc: 0.4170 - val_loss: 1.7200 - val_acc: 0.4073
Epoch 2/130
1093/1093 [==============================] - 28s 26ms/step - loss: 1.3994 - acc: 0.5093 - val_loss: 1.7279 - val_acc: 0.4278
Epoch 3/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.2900 - acc: 0.5493 - val_loss: 1.3876 - val_acc: 0.5018
Epoch 4/130
1093/1093 [==============================] - 28s 26ms/step - loss: 1.2172 - acc: 0.5731 - val_loss: 1.3081 - val_acc: 0.5485
Epoch 5/130
1093/1093 [==============================] - 28s 25ms/step - loss: 1.1510 - acc: 0.5963 - val_loss: 1.3132 - val_acc: 0.5494
Epoch 6/130
1093/1093 [==============================] - 28s 26ms/step - loss: 1.1033 - acc: 0.6130 - val_loss: 1.2658 - val_acc: 0.5764
Epoch 7/130
1093/1093 [==============================] - 28s 26ms/step - loss: 1.0544 - acc: 0.6309 - val_loss: 1.3161 - val_acc: 0.5596
Epoch 8/130
1093/1093 [==================

1093/1093 [==============================] - 28s 26ms/step - loss: 0.2899 - acc: 0.9002 - val_loss: 0.7696 - val_acc: 0.7638
Epoch 61/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.2841 - acc: 0.9021 - val_loss: 0.7051 - val_acc: 0.7834
Epoch 62/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.2703 - acc: 0.9071 - val_loss: 0.6950 - val_acc: 0.7790
Epoch 63/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.2672 - acc: 0.9085 - val_loss: 0.7214 - val_acc: 0.7877
Epoch 64/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.2627 - acc: 0.9101 - val_loss: 0.7160 - val_acc: 0.7871
Epoch 65/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.2618 - acc: 0.9102 - val_loss: 0.6579 - val_acc: 0.7960
Epoch 66/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.2536 - acc: 0.9124 - val_loss: 0.7435 - val_acc: 0.7795
Epoch 67/130
1093/1093 [=======================

1093/1093 [==============================] - 28s 26ms/step - loss: 0.1159 - acc: 0.9593 - val_loss: 0.7363 - val_acc: 0.8064
Epoch 120/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.1166 - acc: 0.9605 - val_loss: 0.7822 - val_acc: 0.7964
Epoch 121/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.1101 - acc: 0.9630 - val_loss: 0.8574 - val_acc: 0.7859
Epoch 122/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.1116 - acc: 0.9617 - val_loss: 0.7890 - val_acc: 0.7988
Epoch 123/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.1091 - acc: 0.9631 - val_loss: 0.9015 - val_acc: 0.7683
Epoch 124/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.1103 - acc: 0.9620 - val_loss: 0.8283 - val_acc: 0.7888
Epoch 125/130
1093/1093 [==============================] - 28s 26ms/step - loss: 0.1053 - acc: 0.9640 - val_loss: 0.8336 - val_acc: 0.7898
Epoch 126/130
1093/1093 [================

In [11]:
test_model = load_model('/data/Quan/datasets/cifar10/transfer_idea18_model.hdf5')
score = test_model.evaluate_generator(test_generator, steps=len(test_images), verbose=1)
score

10000/10000 [==============================] - 70s 7ms/step


[0.7753325680182985, 0.8039]